In [1]:
%matplotlib inline
import pandas as pd

In [2]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [3]:
trades = pd.read_csv('Trades.csv')
trades.head()

,Time,Symb,Side,Price,Qty,Route,Broker,Account,Type,Cloid,P / L,Unnamed: 11
0,09:43:10,MU,B,50.99,400,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
1,09:36:04,SQ,S,74.60,244,SMRT,NaN,TRIB5621,Margin,AUTO,24.40,NaN
2,11:12:58,MU,B,50.86,500,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
3,09:35:42,SQ,S,74.72,81,SMRT,NaN,TRIB5621,Margin,AUTO,17.82,NaN
4,11:08:10,SQ,S,74.56,500,SMRT,NaN,TRIB5621,Margin,AUTO,2.50,NaN


In [4]:
len(trades)

30

In [5]:
trades.sort_values('Time').head(10)

,Time,Symb,Side,Price,Qty,Route,Broker,Account,Type,Cloid,P / L,Unnamed: 11
6,09:35:32,SQ,B,74.50,325,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
3,09:35:42,SQ,S,74.72,81,SMRT,NaN,TRIB5621,Margin,AUTO,17.82,NaN
1,09:36:04,SQ,S,74.60,244,SMRT,NaN,TRIB5621,Margin,AUTO,24.40,NaN
28,09:37:05,SQ,B,74.66,300,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
26,09:37:36,SQ,S,74.77,75,SMRT,NaN,TRIB5621,Margin,AUTO,8.25,NaN
22,09:38:45,SQ,S,74.55,225,SMRT,NaN,TRIB5621,Margin,AUTO,-24.75,NaN
0,09:43:10,MU,B,50.99,400,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
24,09:44:32,MU,S,50.85,400,SMRT,NaN,TRIB5621,Margin,AUTO,-56.00,NaN
23,09:45:07,SQ,SS,74.45,500,SMRT,NaN,TRIB5621,Short,AUTO,0.00,NaN
21,09:45:48,SQ,SS,74.26,125,SMRT,NaN,TRIB5621,Short,AUTO,0.00,NaN


In [6]:
lbls = ['Tindex', 'Start', 'Time', 'Symb', 'Side', 'Price', 'Qty','Balance', 'Account', "P / L", 'Sum', 'Duration', 'Name']

In [7]:
for l in lbls :
    if l not in trades.columns :
        trades[l] = ''
# trades.columns
newTrades = trades[lbls]
newTrades.copy()
newTrades.columns

Index(['Tindex', 'Start', 'Time', 'Symb', 'Side', 'Price', 'Qty', 'Balance',
       'Account', 'P / L', 'Sum', 'Duration', 'Name'],
      dtype='object')

In [8]:
nt = newTrades.sort_values(['Symb', 'Time'])
nt

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
0,,,09:43:10,MU,B,50.99,400,,TRIB5621,0.00,,,
24,,,09:44:32,MU,S,50.85,400,,TRIB5621,-56.00,,,
15,,,10:14:49,MU,B,50.44,500,,TRIB5621,0.00,,,
8,,,10:18:36,MU,S,50.34,500,,TRIB5621,-50.00,,,
2,,,11:12:58,MU,B,50.86,500,,TRIB5621,0.00,,,
25,,,11:21:36,MU,S,50.99,125,,TRIB5621,16.25,,,
13,,,11:33:42,MU,S,51.10,187,,TRIB5621,44.88,,,
7,,,11:39:27,MU,S,50.93,188,,TRIB5621,13.16,,,
6,,,09:35:32,SQ,B,74.50,325,,TRIB5621,0.00,,,
3,,,09:35:42,SQ,S,74.72,81,,TRIB5621,17.82,,,


# Need some logic to figure overnight trades
### When the balance does not get to 0 on one or more stocks, Need to reset the next stock. The best time to check after the first sort and stocks are together by Account->Symb->Time

In [9]:
prevBal = 0
for i, row in nt.iterrows():
    if row['Side'] == 'B' :
        nt.at[i,'Balance'] = ((nt.at[i,'Qty']) + prevBal)
        prevBal = ((nt.at[i,'Qty']) + prevBal)
    else :
        nt.at[i,'Balance'] = ((nt.at[i,'Qty'] * -1) + prevBal)
        prevBal = ((nt.at[i,'Qty'] * -1) + prevBal)
                              
    
nt

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
0,,,09:43:10,MU,B,50.99,400,400,TRIB5621,0.00,,,
24,,,09:44:32,MU,S,50.85,400,0,TRIB5621,-56.00,,,
15,,,10:14:49,MU,B,50.44,500,500,TRIB5621,0.00,,,
8,,,10:18:36,MU,S,50.34,500,0,TRIB5621,-50.00,,,
2,,,11:12:58,MU,B,50.86,500,500,TRIB5621,0.00,,,
25,,,11:21:36,MU,S,50.99,125,375,TRIB5621,16.25,,,
13,,,11:33:42,MU,S,51.10,187,188,TRIB5621,44.88,,,
7,,,11:39:27,MU,S,50.93,188,0,TRIB5621,13.16,,,
6,,,09:35:32,SQ,B,74.50,325,325,TRIB5621,0.00,,,
3,,,09:35:42,SQ,S,74.72,81,244,TRIB5621,17.82,,,


In [10]:
nt.head(2)

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
0,,,09:43:10,MU,B,50.99,400,400,TRIB5621,0.0,,,
24,,,09:44:32,MU,S,50.85,400,0,TRIB5621,-56.0,,,


In [11]:
newTrade = True
for i, row in nt.iterrows():
    if newTrade :
        nt.at[i,'Start'] = row['Time']
        newTrade = False
        oldTime = row['Time']
    else :
        nt.at[i, 'Start'] = oldTime
    if row['Balance'] == 0 :
        newTrade = True

nt = nt.sort_values(['Start', 'Time'])
nt

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
6,,09:35:32,09:35:32,SQ,B,74.50,325,325,TRIB5621,0.00,,,
3,,09:35:32,09:35:42,SQ,S,74.72,81,244,TRIB5621,17.82,,,
1,,09:35:32,09:36:04,SQ,S,74.60,244,0,TRIB5621,24.40,,,
28,,09:37:05,09:37:05,SQ,B,74.66,300,300,TRIB5621,0.00,,,
26,,09:37:05,09:37:36,SQ,S,74.77,75,225,TRIB5621,8.25,,,
22,,09:37:05,09:38:45,SQ,S,74.55,225,0,TRIB5621,-24.75,,,
0,,09:43:10,09:43:10,MU,B,50.99,400,400,TRIB5621,0.00,,,
24,,09:43:10,09:44:32,MU,S,50.85,400,0,TRIB5621,-56.00,,,
23,,09:45:07,09:45:07,SQ,SS,74.45,500,-500,TRIB5621,0.00,,,
21,,09:45:07,09:45:48,SQ,SS,74.26,125,-625,TRIB5621,0.00,,,


In [12]:
nt.head()

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
6,,09:35:32,09:35:32,SQ,B,74.50,325,325,TRIB5621,0.00,,,
3,,09:35:32,09:35:42,SQ,S,74.72,81,244,TRIB5621,17.82,,,
1,,09:35:32,09:36:04,SQ,S,74.60,244,0,TRIB5621,24.40,,,
28,,09:37:05,09:37:05,SQ,B,74.66,300,300,TRIB5621,0.00,,,
26,,09:37:05,09:37:36,SQ,S,74.77,75,225,TRIB5621,8.25,,,


In [13]:
TCount = 1
prevEndTrade = -1
for i, row in nt.iterrows():
    trade = "Trade " + str(TCount)
    if prevEndTrade == 0 :
        TCount = TCount + 1
        prevEndTrade = -1
    trade = "Trade " + str(TCount)
    nt.at[i,'Tindex'] = trade 
    if row['Balance'] == 0 :
        prevEndTrade = 0
nt.head()

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
6,Trade 1,09:35:32,09:35:32,SQ,B,74.50,325,325,TRIB5621,0.00,,,
3,Trade 1,09:35:32,09:35:42,SQ,S,74.72,81,244,TRIB5621,17.82,,,
1,Trade 1,09:35:32,09:36:04,SQ,S,74.60,244,0,TRIB5621,24.40,,,
28,Trade 2,09:37:05,09:37:05,SQ,B,74.66,300,300,TRIB5621,0.00,,,
26,Trade 2,09:37:05,09:37:36,SQ,S,74.77,75,225,TRIB5621,8.25,,,


In [14]:
tradeTotal = 0.0
for i, row in nt.iterrows():
    if row['Balance'] != 0 :
        tradeTotal = tradeTotal + row['P / L']
    else :
        sumtotal = tradeTotal + row['P / L']
        nt.at[i, 'Sum'] = sumtotal
        tradeTotal = 0
nt.head()

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
6,Trade 1,09:35:32,09:35:32,SQ,B,74.50,325,325,TRIB5621,0.00,,,
3,Trade 1,09:35:32,09:35:42,SQ,S,74.72,81,244,TRIB5621,17.82,,,
1,Trade 1,09:35:32,09:36:04,SQ,S,74.60,244,0,TRIB5621,24.40,42.22,,
28,Trade 2,09:37:05,09:37:05,SQ,B,74.66,300,300,TRIB5621,0.00,,,
26,Trade 2,09:37:05,09:37:36,SQ,S,74.77,75,225,TRIB5621,8.25,,,


In [15]:
import datetime

In [16]:
tradeTotal = 0.0
for i, row in nt.iterrows():
    if row['Balance'] == 0 :
        timeEnd = row['Time']
        timeStart = row['Start']
        end=timeEnd.split(":")
        start=timeStart.split(":")
        diff = datetime.datetime(1,1,1,int(end[0]), int(end[1]), int(end[2])) - datetime.datetime(1,1,1,int(start[0]), int(start[1]), int(start[2]))
        nt.at[i,'Duration'] = diff
nt.head()

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
6,Trade 1,09:35:32,09:35:32,SQ,B,74.50,325,325,TRIB5621,0.00,,,
3,Trade 1,09:35:32,09:35:42,SQ,S,74.72,81,244,TRIB5621,17.82,,,
1,Trade 1,09:35:32,09:36:04,SQ,S,74.60,244,0,TRIB5621,24.40,42.22,0:00:32,
28,Trade 2,09:37:05,09:37:05,SQ,B,74.66,300,300,TRIB5621,0.00,,,
26,Trade 2,09:37:05,09:37:36,SQ,S,74.77,75,225,TRIB5621,8.25,,,


In [17]:
for i, row in nt.iterrows():
    longShort = " Long"
    if row['Balance'] == 0 :
        if row['Side'] == 'B' :
            longShort = " Short"
        nt.at[i, 'Name'] = row['Symb'] + longShort
nt.head()

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
6,Trade 1,09:35:32,09:35:32,SQ,B,74.50,325,325,TRIB5621,0.00,,,
3,Trade 1,09:35:32,09:35:42,SQ,S,74.72,81,244,TRIB5621,17.82,,,
1,Trade 1,09:35:32,09:36:04,SQ,S,74.60,244,0,TRIB5621,24.40,42.22,0:00:32,SQ Long
28,Trade 2,09:37:05,09:37:05,SQ,B,74.66,300,300,TRIB5621,0.00,,,
26,Trade 2,09:37:05,09:37:36,SQ,S,74.77,75,225,TRIB5621,8.25,,,


In [18]:
# Temporary place to save and use this chart
# nt.to_csv("Trades.Wednesday.csv")

In [19]:
def addRows(dframe, numCol) :
    ll=list()
    r=list()
    for i in range(len(dframe.columns)) :
        r.append('')
        
    for i in range(numCol) :
        ll.append(r)
    newdf= pd.DataFrame(ll, columns=dframe.columns)
    
    try :
        assert(len(ll[0]) == len(dframe.columns))
    except AssertionError as ex :
        print("Coding error. Assertion is untrue:", ex)
        newdf="Data Frame is not initialized"
        return newdf
    
    dframe = dframe.append(newdf, ignore_index=True)
    
    return dframe

    

In [20]:
nt=addRows(nt,1)
nt.tail(10)

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
21,Trade 7,10:14:49,10:18:36,MU,S,50.34,500,0,TRIB5621,-50,-50,0:03:47,MU Long
22,Trade 8,10:34:58,10:34:58,SQ,B,74.72,500,500,TRIB5621,0,,,
23,Trade 8,10:34:58,10:57:57,SQ,B,74.39,500,1000,TRIB5621,0,,,
24,Trade 8,10:34:58,11:08:10,SQ,S,74.56,500,500,TRIB5621,2.5,,,
25,Trade 8,10:34:58,11:28:42,SQ,S,74.64,500,0,TRIB5621,42.5,45,0:53:44,SQ Long
26,Trade 9,11:12:58,11:12:58,MU,B,50.86,500,500,TRIB5621,0,,,
27,Trade 9,11:12:58,11:21:36,MU,S,50.99,125,375,TRIB5621,16.25,,,
28,Trade 9,11:12:58,11:33:42,MU,S,51.1,187,188,TRIB5621,44.88,,,
29,Trade 9,11:12:58,11:39:27,MU,S,50.93,188,0,TRIB5621,13.16,74.29,0:26:29,MU Long
30,,,,,,,,,,,,,


### This will blow up from wrong types if there is some kind of anomaly. The blank rows in Sum col are str and the filled rows are float. Don't fix it until it blows up. (or you need change this 'now' to production code for some reason) That way we have some case to fix.
### Also need to fix below if the lastCol for Balance > 0 instead of what's there

In [21]:
count=0
tot=0.0
tot2 = 0.0
for i, row in nt.iterrows():
    count=count+1
    if count < len(nt) :
        tot=tot+row['P / L']
        if row['Balance'] == 0 :
            assert(type(row['Sum']) == type(1.1))
            tot2 = tot2 + row['Sum']
        if count == len(nt) -1 :
            lastCol = row['P / L']
            assert(type(lastCol) == type(1.1))
            print("Last col?", row['P / L'])
#         print(count, ": ", tot, tot2)
    else :
        nt.at[i, 'P / L'] = tot
        nt.at[i, 'Sum'] = tot2
#         print("Grand total: ", tot, tot2)

# if(tot != tot2) or lastCol > 0:

if lastCol > 0:

    print('''
    I believe you either closed an overnight trade or you 
    began or changed a previous trade.  Your on your own to 
    fix that. I am going to blithly continue as if nothing 
    happened :)
    ''')
nt.tail()



Last col? 13.16

    I believe you either closed an overnight trade or you 
    began or changed a previous trade.  Your on your own to 
    fix that. I am going to blithly continue as if nothing 
    happened :)
    


,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
26,Trade 9,11:12:58,11:12:58,MU,B,50.86,500,500,TRIB5621,0,,,
27,Trade 9,11:12:58,11:21:36,MU,S,50.99,125,375,TRIB5621,16.25,,,
28,Trade 9,11:12:58,11:33:42,MU,S,51.1,187,188,TRIB5621,44.88,,,
29,Trade 9,11:12:58,11:39:27,MU,S,50.93,188,0,TRIB5621,13.16,74.29,0:26:29,MU Long
30,,,,,,,,,,74.76,74.76,,


In [22]:
ll=list()
r=list()
for i in range(len(nt.columns)) :
    r.append('')

ll.append(r)
ll.append(r)
ll

[['', '', '', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', '', '', '']]

In [23]:
# len(nt.columns)
# l=list(('','','','','','','','','','','','',''))
# ll=list()
# ll.append(l)
# for i in range (19) :
#     ll.append(l)
# nnt = pd.DataFrame(ll, columns=nt.columns)
# nt.append(nnt,ignore_index=True)

In [24]:
ldf = list()
count=0
tot=0.0
tot2 = 0.0
tempdf = pd.DataFrame()
for i, row in nt.iterrows():
    count=count+1
    if count < len(nt) :
        tot=tot+row['P / L']
        if row['Balance'] == 0 :
            assert(type(row['Sum']) == type(1.1))
            tot2 = tot2 + row['Sum']
        if count == len(nt) -1 :
            lastCol = row['P / L']
            assert(type(lastCol) == type(1.1))
            print("Last col?", row['P / L'])
#         print(count, ": ", tot, tot2)
    else :
        nt.at[i, 'P / L'] = tot
        nt.at[i, 'Sum'] = tot2
#         print("Grand total: ", tot, tot2)

# if(tot != tot2) or lastCol > 0:

if lastCol > 0:

    print('''
    I believe you either closed an overnight trade or you 
    began or changed a previous trade.  Your on your own to 
    fix that. I am going to blithly continue as if nothing 
    happened :)
    ''')
nt.tail()



Last col? 13.16

    I believe you either closed an overnight trade or you 
    began or changed a previous trade.  Your on your own to 
    fix that. I am going to blithly continue as if nothing 
    happened :)
    


,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
26,Trade 9,11:12:58,11:12:58,MU,B,50.86,500,500,TRIB5621,0,,,
27,Trade 9,11:12:58,11:21:36,MU,S,50.99,125,375,TRIB5621,16.25,,,
28,Trade 9,11:12:58,11:33:42,MU,S,51.1,187,188,TRIB5621,44.88,,,
29,Trade 9,11:12:58,11:39:27,MU,S,50.93,188,0,TRIB5621,13.16,74.29,0:26:29,MU Long
30,,,,,,,,,,74.76,74.76,,


In [25]:
ldf = list()
count = 1
while True :
    tradeStr = "Trade " + str(count)
    count = count + 1
    tdf = nt[nt.Tindex == tradeStr]
    if len(tdf) > 0 :
        ldf.append(tdf)
    else :
        break
len(ldf)


9

In [26]:
nt = addRows(nt, 2)
nt.tail()

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
28,Trade 9,11:12:58,11:33:42,MU,S,51.1,187,188,TRIB5621,44.88,,,
29,Trade 9,11:12:58,11:39:27,MU,S,50.93,188,0,TRIB5621,13.16,74.29,0:26:29,MU Long
30,,,,,,,,,,74.76,74.76,,
31,,,,,,,,,,,,,
32,,,,,,,,,,,,,


In [27]:
for tdf in ldf :
    nt = nt.append(tdf, ignore_index = True)
    nt = addRows(nt, 20)
    print(count, ": ", len(nt))

11 :  56
11 :  79
11 :  101
11 :  124
11 :  147
11 :  173
11 :  195
11 :  219
11 :  243


In [28]:
name=datetime.date.today().strftime("Trades_%A_%m%d.csv")
nt.to_csv(name)

In [29]:
nt

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
0,Trade 1,09:35:32,09:35:32,SQ,B,74.5,325,325,TRIB5621,0,,,
1,Trade 1,09:35:32,09:35:42,SQ,S,74.72,81,244,TRIB5621,17.82,,,
2,Trade 1,09:35:32,09:36:04,SQ,S,74.6,244,0,TRIB5621,24.4,42.22,0:00:32,SQ Long
3,Trade 2,09:37:05,09:37:05,SQ,B,74.66,300,300,TRIB5621,0,,,
4,Trade 2,09:37:05,09:37:36,SQ,S,74.77,75,225,TRIB5621,8.25,,,
5,Trade 2,09:37:05,09:38:45,SQ,S,74.55,225,0,TRIB5621,-24.75,-16.5,0:01:40,SQ Long
6,Trade 3,09:43:10,09:43:10,MU,B,50.99,400,400,TRIB5621,0,,,
7,Trade 3,09:43:10,09:44:32,MU,S,50.85,400,0,TRIB5621,-56,-56,0:01:22,MU Long
8,Trade 4,09:45:07,09:45:07,SQ,SS,74.45,500,-500,TRIB5621,0,,,
9,Trade 4,09:45:07,09:45:48,SQ,SS,74.26,125,-625,TRIB5621,0,,,


In [43]:
nt

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
6,Trade 1,09:35:32,09:35:32,SQ,B,74.50,325,325,TRIB5621,0.00,,,
3,Trade 1,09:35:32,09:35:42,SQ,S,74.72,81,244,TRIB5621,17.82,,,
1,Trade 1,09:35:32,09:36:04,SQ,S,74.60,244,0,TRIB5621,24.40,42.22,0:00:32,SQ Long
28,Trade 2,09:37:05,09:37:05,SQ,B,74.66,300,300,TRIB5621,0.00,,,
26,Trade 2,09:37:05,09:37:36,SQ,S,74.77,75,225,TRIB5621,8.25,,,
22,Trade 2,09:37:05,09:38:45,SQ,S,74.55,225,0,TRIB5621,-24.75,-16.5,0:01:40,SQ Long
0,Trade 3,09:43:10,09:43:10,MU,B,50.99,400,400,TRIB5621,0.00,,,
24,Trade 3,09:43:10,09:44:32,MU,S,50.85,400,0,TRIB5621,-56.00,-56,0:01:22,MU Long
23,Trade 4,09:45:07,09:45:07,SQ,SS,74.45,500,-500,TRIB5621,0.00,,,
21,Trade 4,09:45:07,09:45:48,SQ,SS,74.26,125,-625,TRIB5621,0.00,,,
